# 📌 Aula 10: Criando um Chatbot Empresarial Pronto para Produção

Este notebook implementa um chatbot seguro, escalável e pronto para uso em empresas.

In [ ]:

import openai
import jwt
import logging
import sqlite3
import requests
from flask import Flask, request, jsonify

# Configuração do log
logging.basicConfig(level=logging.INFO)

# Configuração do Flask
app = Flask(__name__)

# Configuração do banco de dados
conn = sqlite3.connect('chatbot.db', check_same_thread=False)
cursor = conn.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS conversas (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    usuario TEXT,
    mensagem TEXT,
    resposta TEXT,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
)''')
conn.commit()

# Função para gerar tokens JWT
def gerar_token(usuario):
    token = jwt.encode({"user": usuario}, "chave_secreta", algorithm="HS256")
    return token

# Função para armazenar conversas
def salvar_conversa(usuario, mensagem, resposta):
    cursor.execute("INSERT INTO conversas (usuario, mensagem, resposta) VALUES (?, ?, ?)", (usuario, mensagem, resposta))
    conn.commit()

# Função para chamada à OpenAI API
def obter_resposta_openai(mensagem):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "system", "content": "Você é um assistente empresarial."},
                  {"role": "user", "content": mensagem}]
    )
    return response['choices'][0]['message']['content']

# Endpoint para o chatbot
@app.route("/chat", methods=["POST"])
def chat():
    data = request.json
    usuario = data.get("usuario")
    mensagem = data.get("mensagem")
    
    if not usuario or not mensagem:
        return jsonify({"erro": "Usuário e mensagem são obrigatórios"}), 400
    
    resposta = obter_resposta_openai(mensagem)
    salvar_conversa(usuario, mensagem, resposta)
    
    return jsonify({"resposta": resposta})

# Rodando a aplicação Flask
if __name__ == "__main__":
    app.run(debug=True)
